In [1]:
# CNN 구현

# 숙제

# 1. Conv2D Backward 구현
# Forward 에서 쓴 im2col을 이용해 gradient 계산
# pytorch nn.conv2d와 비교해서 상대오차 < 1e-3 달성

# 2. Maxpool 구현 (forward, backward)
# forward : 영역 내 최댓값 / backward : 최댓값 위치로 gradient 전달
# pytorch nn.maxpool2d와 비교 

In [2]:
import numpy as np

In [3]:
# im2col : 입력 이미지(다차원 데이터)를 컬럼 형태로 변환
# filter_h / filter_w : 필터의 높이와 넓이
# stride : n*n의 filter 크기를 지정.
# pad : padding 크기
def im2col(input_data, filter_h, filter_w, stride = 1, pad = 0):
    # 배치 크기, 채널, height, width
    N, C, H, W = input_data.shape # 각 data의 shape를 저장
    out_h = (H + 2 * pad - filter_h) // stride + 1 # 필터가 이동할 위치의 세로
    out_w = (W + 2 * pad - filter_w) // stride + 1 # 필터가 이동할 위치의 가로

    # np.pad : 배열의 경계에 원하는 크기의 패딩을 추가
    # np.pad(array, pad_width, mode='constant', **kwargs)
    # pad_width = [(0,0), (0,0), (pad, pad), (pad, pad)]
    # (1,1)일 경우, 양쪽에 일정하게 1개씩 패딩을 추가한다는 뜻.
    # 여기에서는 input_data의 크기가 (1, 1, 28, 28) 이니까 28*28에는 각각 pad 만큼을 양쪽에 padding 추가함.
    # 그러면 input_data 의 크기가 (1+(0*2), 1+(0*2), 28+(pad*2), 28+(pad*2))가 됨.
    # 각각의 앞뒤에 n씩 더하는 거니까 2를 곱해야 함. 
    # mode: 패딩 방법. 기본값은 'constant'. 패딩을 일정 값으로 채움.
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # input data를 numpy로 생성
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w)) # 변환된 데이터를 담기 위한 임시 변수 

    for y in range(filter_h): # filter 높이 크기만큼 for 
        y_max = y + stride * out_h # y의 슬라이싱 종료 위치. 
        for x in range(filter_w): # filter 넓이 의 크기만큼 for
            x_max = x + stride * out_w # x의 슬라이싱 종료 위치.
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
            # [:, :, y:y_max:stride, ...] y번째 행과 겹치는 모든 픽셀들을 한 번에 가져옴.
            # y:y_max 의 크기를 stride 만큼 건너 뛰어가며 잘라 넣음.
            # x:x_max 의 크기와 stride도 마찬가지.
            # 슬라이싱 시작부터 종료까지 stride 만큼 건너뛰면서 보는 셈.

    print("변환 전 ", col.shape)
    # (N, out_h, out_w, C, filter_h, filter_w) 크기로 바꿔야 함.
    col = col.transpose(0, 4, 5, 1, 2, 3)
    # TODO: gradient 계산 추가
    print("transpose 변환 후 ", col.shape)
    col = col.reshape(N * out_h * out_w , -1) # col의 크기를 변환
    print("reshape 변환 후 ", col.shape)
    return col

# convolution forward
def conv_forward(x, W, b, stride = 1, pad = 0):
    FN, C, FH, FW = W.shape
    N, _, H, W_ = x.shape
    out_h = (H + 2 * pad - FH) // stride + 1
    out_w = (W_ + 2 * pad - FW) // stride + 1

    print("입력 데이터 shape ", x.shape)
    col = im2col(x, FH, FW, stride, pad)
    print("im2col 후 ", col.shape)
    col_W = W.reshape(FN, -1).T
    out = np.dot(col, col_W) + b
    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
    return out

In [73]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# MNIST 불러오기
transform = transforms.Compose([transforms.ToTensor()])
mnist = datasets.MNIST(root="./train_data", train=True, download=True, transform=transform)

# 샘플 이미지 확인
x, y = mnist[0]
print("Label : ", y)
print("Shape : ", x.shape)  # torch.Size([1,28,28])

# numpy 변환 후 convolution 적용
x_np = x.unsqueeze(0).numpy()  # (1,1,28,28)
W = np.random.randn(3, 1, 3, 3) * 0.01 # 아무 숫자나 넣은 거임.
b = np.zeros(3)

out = conv_forward(x_np, W, b, stride=1, pad=1)
                     
print("Conv output shape:", out.shape)  # (1,3,28,28)

Label :  5
Shape :  torch.Size([1, 28, 28])
입력 데이터 shape  (1, 1, 28, 28)
변환 전  (1, 1, 3, 3, 28, 28)
transpose 변환 후  (1, 28, 28, 1, 3, 3)
reshape 변환 후  (784, 9)
im2col 후  (784, 9)
Conv output shape: (1, 3, 28, 28)


In [ ]:
import torch.nn as nn
import torch

# pytorch nn.conv2d 사용해서 비교
def conv_forward2(x, W, b, stride = 1, pad = 0):
    # tensor로 변환
    x_t = torch.from_numpy(x).float()
    W_t = torch.from_numpy(W).float()
    b_t = torch.from_numpy(b).float()
    # weight tensor shape : out channel, in channel, kernel height, kernel width
    o_C, i_C, k_h, k_w = W_t.shape
    
    # kernel size에는 height가 들어가야 함.
    # 왜지?!!!?!?!?
    conv = nn.Conv2d(in_channels = i_C, out_channels = o_C, 
                     kernel_size = k_h, stride = stride, 
                     padding = pad)
    return conv(x_t)

out2 = conv_forward2(x_np, W, b, stride=1, pad=1)
print("Conv output shape:", out2.shape)

Conv output shape: torch.Size([1, 3, 28, 28])
